# Five-area DEC RevBayes

https://revbayes.github.io/tutorials/biogeo/biogeo_simple.html

![schematic](Ortho_earth_scenarios.pdf "schematic")

### Set working directory and input and output filenames

In [2]:
getwd()
#setwd("/Users/cdoorenweerd/Documents/StackIP/Teaching and courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography")

range_fn = "saturnia.n5.range.nex"
tree_fn = "20180606_Saturnia_BEAST_constrained2_ingroup_nocopaxa.nwk"
dist_fn = "saturnia.n5.distances.txt"
connectivity_fn = "saturnia.n5.connectivity.1.txt"

out_fn = "five_area_DEC"
out_state_fn = out_fn + ".states.log"
out_tree_fn = out_fn + ".tre"
out_mcc_fn = out_fn + ".mcc.tre"

mvi = 1
mni = 1

   /Users/cdoorenweerd/Documents/StackIP/Teaching and courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography


### Read in geographical ranges

In [3]:
dat_range_01 = readDiscreteCharacterData(range_fn) # read character data as binary presence-absence
n_areas = dat_range_01.nchar() # record total number of areas
dat_range_01

max_areas <- 2
n_states <- 0
for (k in 0:max_areas) n_states += choose(n_areas, k)

dat_range_n = formatDiscreteCharacterData(dat_range_01, "DEC", n_states) # encode ranges into natural numbers for one character
n_areas
n_states

   Successfully read one character matrix from file 'saturnia.n5.range.nex'

   Standard character matrix with 33 taxa and 5 characters
   Origination:                   saturnia.n5.range.nex
   Number of taxa:                33
   Number of included taxa:       33
   Number of characters:          5
   Number of included characters: 5
   Datatype:                      Standard

   5
   16


Write legend for state labels and range states to file

In [4]:
state_desc = dat_range_n.getStateDescriptions()

state_desc_str = "state,range\n"
for (i in 1:state_desc.size()){state_desc_str += (i-1) + "," + state_desc[i] + "\n"}
write(state_desc_str, file=out_fn+".state_labels.txt")

### Read in dated molecular phylogenetic tree

In [5]:
tree <- readTrees(tree_fn)[1]
tree

   Attempting to read the contents of file "20180606_Saturnia_BEAST_constrained2_ingroup_nocopaxa.nwk"
   Successfully read file
  
   (((((((((((SARBA615_09_Rinaca_chinensis[&index=33]:3.513580,SARBC471_10_Rinaca_ruda_HQ973326[&index=32]:3.513580)[&index=34]:1.239046,SAUPA465_10_Rinaca_naumanni_JN278744[&index=31]:4.752626)[&index=35]:4.110972,(SARBA578_09_Rinaca_minshanensis[&index=30]:4.057900,SARBC468_10_Rinaca_florianii_HQ973323[&index=29]:4.057900)[&index=36]:4.805698)[&index=37]:0.625938,(((SARBA597_09_Rinaca_fujiana[&index=28]:1.349058,SARBA613_09_Rinaca_shaanxiana[&index=27]:1.349058)[&index=38]:3.796523,SARBE1506_13_Rinaca_dusii[&index=26]:5.145581)[&index=39]:1.342367,act16_Rinaca_jonasii[&index=25]:6.487948)[&index=40]:3.001588)[&index=41]:1.296271,act18_Rinaca_boisduvalii[&index=24]:10.785807)[&index=42]:0.391241,SARBB798_09_Rinaca_thibeta_GU664031[&index=23]:11.177048)[&index=43]:2.703537,(SASNB457_09_Rinaca_lesoudieri_GU702972[&index=22]:6.207512,SAWNA102_09_Rinaca_zulei

### Build DEC rate matrices and transition probability matrix

In [6]:
# read connectivity matrix
connectivity <- readDataDelimitedFile(file=connectivity_fn, delimiter=" ")

# read distances matrix
distances <- readDataDelimitedFile(file=dist_fn, delimiter=" ")

# parameter for arrival rate of anagenetic range evolution events
log10_rate_bg ~ dnUniform(-4,2)
log10_rate_bg.setValue(-2)
rate_bg := 10^log10_rate_bg
moves[mvi++] = mvSlide(log10_rate_bg, weight=4)

# dispersal rate matrix
dispersal_rate <- 1.0 # fix base rate of dispersal
distance_scale ~ dnUnif(0,20)
distance_scale.setValue(0.01)
moves[mvi++] = mvScale(distance_scale, weight=3)

for (j in 1:n_areas) {
  for (k in 1:n_areas) {
    dr[j][k] <- 0.0
    if (connectivity[j][k] > 0) {
      dr[j][k]  := dispersal_rate * exp(-distance_scale * distances[j][k])
    }
  }
}

# prior distribution for the relative extirpation rate and assign a move
log_sd <- 0.5
log_mean <- ln(1) - 0.5*log_sd^2
extirpation_rate ~ dnLognormal(mean=log_mean, sd=log_sd)
moves[2] = mvScale(extirpation_rate, weight=2)

for (j in 1:n_areas) {
  for (k in 1:n_areas) {
    er[j][k] <- 0.0
  }
  er[j][j] := extirpation_rate
}

# unify objects
Q_DEC := fnDECRateMatrix(dispersalRates=dr, extirpationRates=er, maxRangeSize=max_areas) # relative matrix

# cladogenetic transition probability matrix
clado_event_types <- [ "s", "a" ]
clado_event_probs <- simplex(1, 1)
P_DEC := fnDECCladoProbs(eventProbs=clado_event_probs,
                            eventTypes=clado_event_types,
                            numCharacters=n_areas,
                            maxRangeSize=max_areas)

# encapsulate all model components
m_bg ~ dnPhyloCTMCClado(tree=tree,
                           Q=Q_DEC,
                           cladoProbs=P_DEC,
                           branchRates=rate_bg,
                           nSites=1,
                           type="NaturalNumbers")

# attach observed ranges to the model
m_bg.clamp(dat_range_n)

### Run model

In [7]:
# add monitors that keep track of the MCMC
monitors[1] = mnScreen(rate_bg, extirpation_rate, printgen=100)
monitors[2] = mnModel(file=out_fn+".params.log", printgen=10)
monitors[3] = mnFile(tree, file=out_fn+".tre", printgen=10)
monitors[4] = mnJointConditionalAncestralState(tree=tree,
                                                    ctmc=m_bg,
                                                    filename=out_fn+".states.log",
                                                    type="NaturalNumbers",
                                                    printgen=10,
                                                    withTips=true,
                                                    withStartStates=true)

In [8]:
mymodel = model(m_bg)

In [9]:
mymcmc = mcmc(mymodel, moves, monitors)

In [10]:
mymcmc.run(25000)


   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 2 different moves in a random move schedule with 6 moves per iteration

Iter        |      Posterior   |     Likelihood   |          Prior   |   extirpatio..   |        rate_bg   |    elapsed   |        ETA   |
------------------------------------------------------------------------------------------------------------------------------------------
0           |       -32.0265   |       -26.8458   |       -5.18067   |      0.4352519   |           0.01   |   00:00:00   |   --:--:--   |
100         |       -33.5182   |       -24.5775   |       -8.94071   |       2.916074   |    0.006498007   |   00:00:15   |   --:--:--   |
200         |       -32.4675   |       -27.2259   |       -5.24157   |       1.120778   |     0.01409674   |   00:00:29   |   00:59:56   |
300         |       -34.1265   |       -29.3623   |       -4.76423   |      0.6725031   |   0.0002967562   |   00:00:44   |   01:00:22

5100        |       -30.7241   |       -25.2031   |       -5.52097   |       1.271886   |    0.005505396   |   00:12:20   |   00:48:07   |
5200        |       -30.4665   |       -25.6942   |       -4.77225   |       0.734896   |    0.003144038   |   00:12:35   |   00:47:54   |
5300        |        -31.827   |        -24.897   |          -6.93   |       1.946134   |    0.006084039   |   00:12:50   |   00:47:42   |
5400        |       -30.4524   |       -25.4337   |       -5.01868   |      0.9825063   |    0.006013165   |   00:13:04   |   00:47:25   |
5500        |       -32.8028   |       -26.9642   |       -5.83856   |       1.430806   |    0.001005006   |   00:13:18   |   00:47:09   |
5600        |       -35.8767   |       -29.2988   |       -6.57788   |       1.781617   |     0.02261939   |   00:13:34   |   00:46:59   |
5700        |       -30.3857   |       -25.6192   |       -4.76643   |      0.6605593   |    0.004140685   |   00:13:49   |   00:46:46   |
5800        |       -30.472

10300       |       -31.3151   |       -25.3247   |       -5.99041   |       1.504263   |    0.003025304   |   00:24:44   |   00:35:17   |
10400       |       -30.7726   |       -25.5728   |       -5.19978   |       1.096554   |     0.00284201   |   00:24:58   |   00:35:02   |
10500       |       -30.3165   |       -25.5307   |       -4.78576   |      0.7641557   |    0.004947464   |   00:25:13   |   00:34:49   |
10600       |       -30.8578   |       -25.2087   |       -5.64903   |       1.337078   |    0.003944064   |   00:25:28   |   00:34:35   |
10700       |       -31.8182   |       -26.7726   |       -5.04561   |       1.000715   |    0.001205366   |   00:25:43   |   00:34:22   |
10800       |       -31.1382   |        -25.837   |       -5.30115   |       1.154411   |    0.002189694   |   00:25:58   |   00:34:08   |
10900       |        -33.534   |       -26.7939   |        -6.7401   |        1.85742   |    0.001044239   |   00:26:12   |   00:33:53   |
11000       |       -30.535

15700       |       -32.1222   |       -26.1751   |       -5.94705   |      0.3184344   |    0.002459303   |   00:38:35   |   00:22:51   |
15800       |       -30.8748   |       -25.1304   |       -5.74436   |       1.384563   |    0.004647424   |   00:38:49   |   00:22:36   |
15900       |       -30.9116   |       -25.5907   |       -5.32093   |       1.165367   |     0.00269898   |   00:39:04   |   00:22:21   |


Iter        |      Posterior   |     Likelihood   |          Prior   |   extirpatio..   |        rate_bg   |    elapsed   |        ETA   |
------------------------------------------------------------------------------------------------------------------------------------------
16000       |       -30.9333   |       -25.4809   |       -5.45239   |       1.236131   |    0.002913951   |   00:39:18   |   00:22:06   |
16100       |       -32.8709   |       -25.3692   |       -7.50173   |       2.214745   |    0.002344037   |   00:39:34   |   00:21:52   |
16200       |       -31.2

20900       |       -31.0789   |       -25.1274   |       -5.95147   |       1.485535   |    0.006153719   |   00:51:39   |   00:10:07   |
21000       |       -31.9125   |       -26.9958   |       -4.91669   |      0.5210261   |    0.001158771   |   00:51:55   |   00:09:53   |
21100       |       -34.0384   |       -26.1934   |       -7.84495   |       2.378104   |     0.01350809   |   00:52:09   |   00:09:38   |
21200       |       -32.2488   |       -24.7853   |       -7.46357   |         2.1967   |     0.00595893   |   00:52:23   |   00:09:23   |
21300       |       -32.6431   |       -24.8343   |       -7.80877   |       2.360788   |    0.007132783   |   00:52:38   |   00:09:08   |
21400       |       -33.2019   |       -26.8785   |       -6.32342   |      0.2841608   |    0.001341265   |   00:52:54   |   00:08:53   |
21500       |       -30.7582   |       -25.9373   |       -4.82087   |      0.5800218   |    0.002630958   |   00:53:09   |   00:08:39   |
21600       |       -30.631

### Write results to output files

In [11]:
tree_trace = readTreeTrace(file=out_tree_fn, treetype="clock")
tree_trace.setBurnin(0.25) # gratuitous when phylogeny is fixed
n_burn = tree_trace.getBurnin()

mcc_tree = mccTree(tree_trace, file=out_mcc_fn)
state_trace = readAncestralStateTrace(file=out_state_fn)
tree_trace = readAncestralStateTreeTrace(file=out_tree_fn, treetype="clock")

anc_tree = ancestralStateTree(tree=mcc_tree,
                              ancestral_state_trace_vector=state_trace,
                              tree_trace=tree_trace,
                              include_start_states=true,
                              file=out_tree_fn+".ase.tre",
                              burnin=n_burn,
                              site=1)

   Processing file "/Users/cdoorenweerd/Documents/StackIP/Teaching and
   courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography/five_area_DEC.tre"

Progress:
0---------------25---------------50---------------75--------------100
********************************************************************

   Compiling maximum clade credibility tree from 2501 trees in tree trace, using a burnin of 625 trees.
   Summarizing clades ...

Progress:
0---------------25---------------50---------------75--------------100
********************************************************************

   Annotating tree ...
Processing file "/Users/cdoorenweerd/Documents/StackIP/Teaching and courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography/five_area_DEC.states.log"
Processing file "/Users/cdoorenweerd/Documents/StackIP/Teaching and courses/2019_Systematics/2019_fall_UH_Systematics_class/20191008_revbayes_biogeography/five_area_DEC.tre"
   Compili

### Calculate marginal likelihood

Using stepping-stone simulation and path sampling

In [12]:
pow_p = powerPosterior(mymodel, moves, monitors, "model.out", cats=50) 
pow_p.burnin(generations=10000,tuningInterval=1000)


Running burn-in phase of Power Posterior sampler for 10000 iterations.
The simulator uses 2 different moves in a random move schedule with 6 moves per iteration


Progress:
0---------------25---------------50---------------75--------------100
********************************************************************


In [12]:
pow_p.run(generations=1000)


Running power posterior analysis ...
Step  1 / 51		****************************************
Step  2 / 51		****************************************
Step  3 / 51		****************************************
Step  4 / 51		****************************************
Step  5 / 51		****************************************
Step  6 / 51		****************************************
Step  7 / 51		****************************************
Step  8 / 51		****************************************
Step  9 / 51		****************************************
Step 10 / 51		****************************************
Step 11 / 51		****************************************
Step 12 / 51		****************************************
Step 13 / 51		****************************************
Step 14 / 51		****************************************
Step 15 / 51		****************************************
Step 16 / 51		****************************************
Step 17 / 51		****************************************
Step 18 / 51		*************

In [13]:
ss = steppingStoneSampler(file="model.out", powerColumnName="power", likelihoodColumnName="likelihood")

   Error:	Could not not file model.out


In [14]:
ss.marginal()

   -27.06624


In [15]:
ps = pathSampler(file="model.out", powerColumnName="power", likelihoodColumnName="likelihood")

In [16]:
ps.marginal() 

   -27.06222
